In [6]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from neo4j import basic_auth
import json
import pandas as pd
from IPython.display import display
import time
import os
from openai import OpenAI
from IPython.display import display

# Configuração do Neo4J
uri = "neo4j://138.2.240.156:7688"
auth = basic_auth("neo4j", "Password1")
db = "neo4j"

# Conexão ao Neo4j
driver = GraphDatabase.driver(uri, auth=auth)

# Function to check connection and list databases
def test_connection():
    try:
        with driver.session() as session:
            result = session.run("SHOW DATABASES YIELD name")
            print("Connection to Neo4j successful!\nAvailable databases:")
            for record in result:
                print(record['name'])
    except Exception as e:
        print(f"Error connecting to Neo4j: {e}")

def run_query(query):
    with driver.session(database=db) as session:
        return result

def run_query_graph(query):
    with driver.session(database=db) as session:
        result = session.run(query)
        graph = result.graph()
        return GraphWidget(graph=graph)

def run_query_data(query):
    """
    Executes a query against the database and returns a DataFrame.
    Records query response time.
    """
    start_time = time.time()
    with driver.session(database=db) as session:
        result = session.run(query)
        data = [record.data() for record in result]
        end_time = time.time()
        duration_ms = (end_time - start_time) * 1000
        print(f"Response time: {duration_ms:.2f} ms")
        return pd.DataFrame(data)

def run_query_data_graph(query):
    with driver.session(database=db) as session:
        result = session.run(query)
        data = [record.data() for record in result]
        graph = result.graph()
        graph_widget = GraphWidget(graph=graph)
        display(graph_widget)
        return pd.DataFrame(data)

def save_to_csv_incrementally(df, filename):
    """
    Saves a DataFrame to a CSV file, adding to the end if the file already exists.
    """
    df.to_csv(filename, mode='a', header=not pd.io.common.file_exists(filename), index=False)

# Test the initial connection
test_connection()

Connection to Neo4j successful!
Available databases:
neo4j
system


In [ ]:
import pandas as pd
import os

# Função para executar uma query no Neo4j
def run_query(query):
    with driver.session(database=db) as session:
        result = session.run(query)
        return [record.data() for record in result]

# Caminho do arquivo CSV de entrada e saída
input_csv = 'no_categorized.csv'  # Arquivo com os IDs dos artefatos
output_folder = 'output_dependencies/'  # Pasta onde os CSVs serão salvos

# Criação da pasta de saída, se não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Leitura do CSV de entrada
ids = pd.read_csv(input_csv)['ID'].tolist()

# Inicializa variáveis para controle
batch_size = 100000  # Quantidade de IDs por arquivo CSV
current_batch = []   # Lista temporária para armazenar os resultados de um batch
batch_count = 1      # Contador para os arquivos gerados

# Processa cada ID do CSV
for idx, artifact_id in enumerate(ids, start=1):
    # Query Cypher para buscar TODAS as dependências do ID
    query = f"MATCH (a:Artifact {{id: '{artifact_id}'}})<-[:dependency]-(dependentArtifact) RETURN DISTINCT dependentArtifact.id AS DependencyArtifactID"
    dependencies = run_query(query)
    
    # Verifica se a query retornou resultados
    if dependencies:
        print(f"ID: {artifact_id} - Dependências encontradas: {len(dependencies)}")
        # Adiciona os resultados ao batch atual
        for dep in dependencies:
            current_batch.append({'ArtifactID': artifact_id, 'DependencyID': dep['DependencyArtifactID']})
    else:
        print(f"ID: {artifact_id} - Nenhuma dependência encontrada.")

    # Verifica se o batch atingiu o tamanho máximo
    if len(current_batch) >= batch_size or idx == len(ids):
        # Salva o batch atual em um arquivo CSV
        output_csv = os.path.join(output_folder, f'batch_{batch_count}_dependencies.csv')
        pd.DataFrame(current_batch).to_csv(output_csv, index=False)
        print(f"Lote {batch_count} salvo em: {output_csv}")
        
        # Limpa o batch atual e incrementa o contador
        current_batch = []
        batch_count += 1


ID: .com.daikit:spring-boot-starter-data-graphql - Dependências encontradas: 5
ID: .com.daikit:spring-boot-starter-data-graphql-jpa - Dependências encontradas: 3
ID: .org.slf4j:slf4j-api - Dependências encontradas: 9
ID: AIUIPlayer:common - Dependências encontradas: 4
ID: ALogger:alog - Dependências encontradas: 2
ID: AR:communication-release - Dependências encontradas: 1
ID: ActivityStarter:annotation - Dependências encontradas: 1
ID: AdCelSDK:adcel-sdk-core - Dependências encontradas: 2
ID: AdManager:mediationsdk-7.1.6 - Dependências encontradas: 1
ID: AdappterAp:data - Dependências encontradas: 1
ID: AdappterAp:unity-ads - Dependências encontradas: 1
ID: Adapter:annotation - Dependências encontradas: 1
ID: Aftermath:aftermath-annotations - Dependências encontradas: 2
ID: AgileDevKt:corekt - Dependências encontradas: 1
ID: AhmerPDF:AhmerPdfium - Dependências encontradas: 1
ID: Alivery:bear - Dependências encontradas: 3
ID: Analytics-Android-Next:core - Dependências encontradas: 1
ID:

In [2]:
import pandas as pd
import os

# Pasta onde os arquivos CSV estão localizados
input_folder = 'output_dependencies/'
output_file = 'consolidated_dependencies.csv'  # Nome do arquivo consolidado

# Lista para armazenar todos os DataFrames
dataframes = []

# Lista de arquivos CSV na pasta
csv_files = [f for f in sorted(os.listdir(input_folder)) if f.endswith('.csv')]

# Total de arquivos CSV
total_files = len(csv_files)

# Itera sobre todos os arquivos na pasta
for idx, filename in enumerate(csv_files, start=1):
    file_path = os.path.join(input_folder, filename)
    print(f"Lendo arquivo: {file_path}")
    df = pd.read_csv(file_path)
    dataframes.append(df)  # Adiciona o DataFrame à lista
    
    # Barra de progresso a cada 25%
    progress = (idx / total_files) * 100
    if progress >= 25 and progress < 50 and idx % (total_files // 4) == 0:
        print("25% concluído...")
    elif progress >= 50 and progress < 75 and idx % (total_files // 2) == 0:
        print("50% concluído...")
    elif progress >= 75 and idx % (3 * total_files // 4) == 0:
        print("75% concluído...")

# Combina todos os DataFrames em um único DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Salva o DataFrame combinado em um único arquivo CSV
combined_df.to_csv(output_file, index=False)
print(f"Arquivo consolidado salvo como: {output_file}")


Lendo arquivo: output_dependencies/batch_01_dependencies.csv
Lendo arquivo: output_dependencies/batch_02_dependencies.csv
Lendo arquivo: output_dependencies/batch_03_dependencies.csv
Lendo arquivo: output_dependencies/batch_04_dependencies.csv
Lendo arquivo: output_dependencies/batch_05_dependencies.csv
Lendo arquivo: output_dependencies/batch_06_dependencies.csv
Lendo arquivo: output_dependencies/batch_07_dependencies.csv
Lendo arquivo: output_dependencies/batch_08_dependencies.csv
Lendo arquivo: output_dependencies/batch_09_dependencies.csv
Lendo arquivo: output_dependencies/batch_10_dependencies.csv
Lendo arquivo: output_dependencies/batch_11_dependencies.csv
Lendo arquivo: output_dependencies/batch_12_dependencies.csv
Lendo arquivo: output_dependencies/batch_13_dependencies.csv
Lendo arquivo: output_dependencies/batch_14_dependencies.csv
Lendo arquivo: output_dependencies/batch_15_dependencies.csv
Lendo arquivo: output_dependencies/batch_16_dependencies.csv
Lendo arquivo: output_de

In [1]:
import pandas as pd

# Nome do arquivo de entrada e saída
input_csv = 'consolidated_dependencies.csv'
output_csv = 'cleaned_consolidated_dependencies.csv'  # Nome do arquivo de saída

# Carrega o arquivo CSV em chunks para lidar com grandes arquivos
chunk_size = 500000  # Tamanho do chunk para leitura em partes
chunks = []

# Processa o arquivo CSV em pedaços
for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    # Remove a parte após o segundo ":" nas colunas ArtifactID e DependencyID
    chunk['ArtifactID'] = chunk['ArtifactID'].apply(lambda x: ':'.join(x.split(':')[:2]))
    chunk['DependencyID'] = chunk['DependencyID'].apply(lambda x: ':'.join(x.split(':')[:2]))
    # Remove duplicatas
    chunk = chunk.drop_duplicates()
    chunks.append(chunk)

# Combina os chunks limpos em um único DataFrame
cleaned_df = pd.concat(chunks, ignore_index=True)

# Salva o DataFrame limpo em um novo arquivo CSV
cleaned_df.to_csv(output_csv, index=False)
print(f"Arquivo limpo salvo como: {output_csv}")


Arquivo limpo salvo como: cleaned_consolidated_dependencies.csv


In [8]:
import pandas as pd

# Caminhos dos arquivos CSV
csv1_file = 'categorized.csv'  # CSV com a lista de IDs
csv2_file = 'consolidate_deps_uniquis.csv'  # CSV onde serão feitas as buscas
output_file = 'TAGS_noCategorizedsNewVersion.csv'  # CSV com o resultado filtrado

# Leitura dos arquivos CSV
csv1 = pd.read_csv(csv1_file)
csv2 = pd.read_csv(csv2_file)

# Criação de um dicionário para mapear IDs às TAGs (para busca rápida)
id_to_tag = dict(zip(csv1['ID'], csv1['TAG']))

#  Filtra o CSV2 verificando se a coluna 'DependencyID' contém algum ID do CSV1 e cria uma cópia explícita
filtered_csv2 = csv2[csv2['DependencyID'].isin(id_to_tag.keys())].copy()

# Adiciona a coluna 'TAG' correspondente do CSV1
filtered_csv2.loc[:, 'TAG'] = filtered_csv2['DependencyID'].map(id_to_tag)

# Salva o resultado em um novo arquivo CSV
filtered_csv2.to_csv(output_file, index=False)
print(f"Arquivo gerado com sucesso: {output_file}")

Arquivo gerado com sucesso: TAGS_noCategorizedsNewVersion.csv
